In [ ]:
dirname(dirname(dirname(getwd())))

In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))
source(paste0(HELP_DIR, "fisher.r"))

In [ ]:
library(patchwork)

In [ ]:
extra_theme <- 
theme(legend.position = "none") + 
  theme(axis.title.x = element_blank(),axis.title.y = element_blank()) + 
  theme(axis.text.y = element_text(size = 12), plot.title = element_text(size = 13)) 

# Read prepared data

- Prepared categorical markers

In [ ]:
ready <- readRDS(paste0(SHARE_DIR, "3_marginal_post_process.Rds"))

In [ ]:
base <- 
ready %>% 
 mu(cohortGo = ifelse(cohortGo != "Pan-Cancer", gsub(" \\(ant\\)agonist", "", cohortGo), "All treatments")) %>% 
 gb(cohortGo, pan, group) %>% 
 su( patients = max(total_patients, na.rm = TRUE),
     fisher = mean(sig_fisher, na.rm = TRUE), 
     pfs = mean(sig_pfs, na.rm = TRUE), 
     `Both P-Adj < .1` = mean(sig_fisher + sig_pfs == 2)) %>% 
 mu( `Fisher Only P-Adj < .1` = fisher - `Both P-Adj < .1`,
     `Cox PH Only P-Adj < .1` = pfs - `Both P-Adj < .1`) %>% 
 se( -fisher, -pfs) %>% 
 ga(gp, val, -cohortGo, -group, -pan, -patients) %>% 
 ug() %>%
 mu( gp = factor(gp, levels = rev(c("Fisher Only P-Adj < .1", "Cox PH Only P-Adj < .1", "Both P-Adj < .1"))))

# 2 - Barplots for different cohorts

In [ ]:
plter <- function(plt_base, title = "X", text_size = 4){
 dd_tot <- plt_base %>% gb(cohortGo, patients) %>% su(tot = sum(val))
 plt_base %>%
  ggplot(aes(y = fct_reorder(cohortGo, patients), x = val, alpha = gp)) + 
  geom_bar(stat = "identity", color = "black", fill = "#e52f28") +   
  go_theme + 
  extra_theme + 
  labs(title = title, alpha = NULL) + 
  scale_x_continuous(labels = scales::percent, expand = expansion(c(0, .3)), limits = c(0,.2)) +
  geom_text(data = dd_tot, aes(y = cohortGo, x = tot, label = paste0(round(signif(100*tot,2),1), "%")), inherit.aes = FALSE, hjust = -.2, size = 4) + 
  scale_alpha_manual(values = c("Both P-Adj < .1" = 1, "Fisher Only P-Adj < .1" = .2,  "Cox PH Only P-Adj < .1" = .6)) 
}

In [ ]:
p1 <- plter(base %>% fi(!pan, group == "treatment"), "Cohort Specific: Drug Treatment") + theme(legend.position = c(0.65, 0.6) ) + labs(fill = NULL)
p2 <- plter(base %>% fi(pan, group == "treatment"), "Pan-Cancer: Drug Treatment")
p3 <- plter(base %>% fi(!pan, group == "mechanism"), "Cohort Specific: Drug Mechanism")
p4 <- plter(base %>% fi(pan, group == "mechanism"), "Pan-Cancer: Drug Mechanism")
p5 <- plter(base %>% fi(!pan, group == "type"), "Cohort Specific: Treatment Type") 
p6 <- plter(base %>% fi(pan, group == "type"), "Pan-Cancer: Treatment Type") 

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 8)

In [ ]:
cohort <- (p1 / p3 / p5 ) + plot_layout(heights = c(5, 7, 8)) 
pan_cancer <- (p2 / p4 / p6 ) + plot_layout(heights = c(6, 6, 2.5)) 
cohorts_full = (cohort | pan_cancer ) + plot_layout(widths = c(1,1)) 

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 11)
cohorts_full

In [ ]:
ggsave( paste0(SHARE_DIR, "2_volcano_summaries_sig_cohorts.png"), 
        plot = cohorts_full, 
        width = 14, 
        height = 11)

In [ ]:
paste0(SHARE_DIR, "2_volcano_summaries_sig_cohorts.png")